In [1]:
import torch
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch import nn, optim
import torch.nn.functional as F

In [2]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [3]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
for param in model.parameters():
    param.requires_grad = False 

In [5]:
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,256)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(256,64)),
    ('output', nn.LogSoftmax(dim=1))
]))
model.fc = fc
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 10
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], axis=1)  # Make image 3 channels
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                images = torch.cat([images, images, images], axis=1) # Make image 3 channels
                
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/10..  Training Loss: 0.960..  Test Loss: 0.850..  Test Accuracy: 0.692
Epoch: 2/10..  Training Loss: 0.825..  Test Loss: 0.800..  Test Accuracy: 0.710
Epoch: 3/10..  Training Loss: 0.777..  Test Loss: 0.809..  Test Accuracy: 0.703
Epoch: 4/10..  Training Loss: 0.754..  Test Loss: 0.805..  Test Accuracy: 0.708
Epoch: 5/10..  Training Loss: 0.736..  Test Loss: 0.815..  Test Accuracy: 0.710
Epoch: 6/10..  Training Loss: 0.732..  Test Loss: 0.775..  Test Accuracy: 0.721
Epoch: 7/10..  Training Loss: 0.714..  Test Loss: 0.777..  Test Accuracy: 0.721
Epoch: 8/10..  Training Loss: 0.705..  Test Loss: 0.794..  Test Accuracy: 0.718
Epoch: 9/10..  Training Loss: 0.698..  Test Loss: 0.814..  Test Accuracy: 0.712
Epoch: 10/10..  Training Loss: 0.694..  Test Loss: 0.769..  Test Accuracy: 0.730


In [8]:
for param in model.parameters():
    param.requires_grad = True 

In [9]:
optimizer = optim.Adam(model.parameters(), lr=3e-5)

epochs = 10
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], axis=1)  # Make image 3 channels
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                images = torch.cat([images, images, images], axis=1) # Make image 3 channels
                
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/10..  Training Loss: 0.481..  Test Loss: 0.449..  Test Accuracy: 0.842
Epoch: 2/10..  Training Loss: 0.340..  Test Loss: 0.380..  Test Accuracy: 0.866
Epoch: 3/10..  Training Loss: 0.282..  Test Loss: 0.344..  Test Accuracy: 0.879
Epoch: 4/10..  Training Loss: 0.242..  Test Loss: 0.339..  Test Accuracy: 0.883
Epoch: 5/10..  Training Loss: 0.210..  Test Loss: 0.332..  Test Accuracy: 0.887
Epoch: 6/10..  Training Loss: 0.188..  Test Loss: 0.335..  Test Accuracy: 0.890
Epoch: 7/10..  Training Loss: 0.166..  Test Loss: 0.319..  Test Accuracy: 0.897
Epoch: 8/10..  Training Loss: 0.147..  Test Loss: 0.331..  Test Accuracy: 0.897
Epoch: 9/10..  Training Loss: 0.132..  Test Loss: 0.333..  Test Accuracy: 0.898
Epoch: 10/10..  Training Loss: 0.120..  Test Loss: 0.362..  Test Accuracy: 0.897
